In [32]:
import pickle as pkl
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<h1 style='text-align:center'>Loading Models</h1>


In [33]:
age_model_file_path = './model_checkpoint_age.h5'
identity_model_file_path = './model_checkpoint_id.h5'
emotion_model_file_path = './model_checkpoint_em.h5'

scaler = '../../Assignment 1/task1/scaler.sav'

In [34]:
def load_compile_model_age(input_shape=(3072,)):
    img_inputs_layer = keras.Input(shape=input_shape)

    hidden_layer = layers.Dense(64,activation='relu')(img_inputs_layer) 

    output = layers.Dense(1,bias_initializer='ones')(hidden_layer)

    # Create the model
    model = tf.keras.models.Model(inputs=img_inputs_layer, outputs=output)
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error')

    return model

In [35]:
def load_compile_model_identity(input_shape=(3072,)):
    img_inputs_layer = keras.Input(shape=input_shape)

    hidden_layer = layers.Dense(64,activation='relu')(img_inputs_layer) 

    hidden_layer_2 = layers.Dense(64,activation='relu')(hidden_layer)
    
    output = layers.Dense(1,activation='sigmoid',bias_initializer='ones')(hidden_layer_2)


    # Create the model
    model = tf.keras.models.Model(inputs=img_inputs_layer, outputs=output)
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    return model

In [36]:
def load_compile_model_emotions(input_shape=(3072,)):
    img_inputs_layer = keras.Input(shape=input_shape)

    hidden_layer = layers.Dense(128,activation='relu')(img_inputs_layer) 

    hidden_layer_2 = layers.Dense(64,activation='relu')(hidden_layer)
    
    
    output = layers.Dense(1,activation='sigmoid',bias_initializer='ones')(hidden_layer_2)


    # Create the model
    model = tf.keras.models.Model(inputs=img_inputs_layer, outputs=output)
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    return model


In [37]:
age_model = load_compile_model_age()
age_model.load_weights(age_model_file_path)
id_model = load_compile_model_identity()
id_model.load_weights(identity_model_file_path)
em_model = load_compile_model_emotions()
em_model.load_weights(emotion_model_file_path)
scaler = pkl.load(open(scaler,'rb'))

In [38]:
model = [age_model,id_model,em_model]

In [43]:
def App(models):
    try:
        vidcap = cv2.VideoCapture(0)
        font = cv2.FONT_HERSHEY_SIMPLEX
        thickness = 2
        font_scale = 1
        if vidcap.isOpened():
            while True:
                ret, frame = vidcap.read()
                if ret:
                    frame_copy = np.array([cv2.resize(frame,(32,32)).flatten()])
                    frame_copy = scaler.transform(frame_copy)
                    age_model = np.round(models[0].predict(frame_copy)[0][0] * 100)
                    identity = models[1].predict(frame_copy)[0]
                    rounded_id = np.round(identity)
                    if rounded_id == 1:
                        id = 'haziq'
                    else:
                        id = 'sulaiman'
                    emotion_value = np.round(models[2].predict(frame_copy)[0])
                    if emotion_value == 1:
                        emotion = 'Smile'
                    else:
                        emotion = 'no-smile'
                    cv2.putText(frame,str(age_model),(0,440),font,font_scale, (255, 0, 0), thickness, cv2.LINE_AA)
                    cv2.putText(frame,id,(500,300),font,font_scale, (0, 0, 255), thickness, cv2.LINE_AA)
                    cv2.putText(frame,str(identity),(500,350),font,font_scale, (0, 0, 255), thickness, cv2.LINE_AA)
                    cv2.putText(frame,emotion,(250,170),font,font_scale, (0, 0, 255), thickness, cv2.LINE_AA)
                    cv2.imshow('Frame', frame)
                else:
                    print("Frame not captured")
                if cv2.waitKey(5) & 0xFF==ord('q'):
                    break
        else:
            print("Video is not working")
        vidcap.release()
        cv2.destroyAllWindows() 
    except Exception as e:
        print(e)

In [44]:
App(model)

1/1 [==============================] - 0s 19ms/step
